Simple demo to test out new `Portfolio` features as well as selecting documents by text.

In [2]:
from datamule import Portfolio, Config

# we can set the default download source for the portfolio to either datamule or sec
# datamule is faster, and has more data, but costs $1/100,000 downloads. SEC is free but rate limited at 5 requests per second.
# sec is the default source
config = Config()
config.set_default_source("datamule") # set default source to datamule
print(f"Default source: {config.get_default_source()}")
portfolio = Portfolio('ingest/test3')

portfolio.download_submissions(ticker='MSFT',submission_type='10-K')

Default source: sec
Loading 28 submissions


Loading submissions: 100%|██████████| 28/28 [00:00<00:00, 150.83it/s]
Progress [Rate: 5.0/s | 0.01 MB/s]: 100%|██████████| 24/24 [00:02<00:00, 11.06it/s]
Progress [Rate: 6.0/s | 6.17 MB/s]: 100%|██████████| 24/24 [00:05<00:00,  4.04it/s]  
